In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import math
import numpy as np
import copy
import torch.utils.data as Data
from torch.autograd import Variable
from torch.optim import Adam,RMSprop
from scipy.linalg import sqrtm
import math
import time
import timeit

# import cv2
# from torchvision import transforms
# from PIL import Image
import numpy as np

In [2]:
#define Autoencoder structure for the wireless sytem model
class CNN_autoencoder(nn.Module):
    def __init__(self, N_bit, Nt, Nr, N_RIS1, N_RIS2, Block_length):
        super(CNN_autoencoder, self).__init__()
        
        self.N_bit = N_bit
        self.Nt = Nt
        self.Nr = Nr
        self.N_RIS1 = N_RIS1
        self.N_RIS2 = N_RIS2
        kernel = 1
        self.Block_length = Block_length
        # sigma = 1e-12
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=self.N_bit,out_channels=256,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(in_channels=256,out_channels=256,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            
            nn.Conv1d(in_channels=256,out_channels=2*Nt,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(2*Nt)
        )
     
        self.decoder = nn.Sequential(
            nn.Conv1d(in_channels=2*self.Nr+2*self.Nt*self.Nr,out_channels=512,kernel_size=kernel,stride=1),
#             nn.Conv1d(in_channels=2*self.Nr,out_channels=256,kernel_size=1,stride=1),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(in_channels=512,out_channels=512,kernel_size=1,stride=kernel),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
            nn.Conv1d(in_channels=512,out_channels=self.N_bit,kernel_size=kernel,stride=1),
            #nn.BatchNorm1d(N_bit*Nt)
            nn.Softmax(1)
        )
        self.RIS1 = nn.Sequential(
#             nn.Conv1d(in_channels=2*self.Nt*N_RIS+2*N_RIS,out_channels=256,kernel_size=1,stride=1),
            nn.Conv1d(in_channels=2*N_RIS1,out_channels=256,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(in_channels=256,out_channels=256,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Conv1d(in_channels=256,out_channels=self.N_RIS1,kernel_size=kernel,stride=1),
            # nn.Tanh()
        )
        self.RIS2 = nn.Sequential(
#             nn.Conv1d(in_channels=2*self.Nt*N_RIS+2*N_RIS,out_channels=256,kernel_size=1,stride=1),
            nn.Conv1d(in_channels=2*N_RIS2,out_channels=256,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(in_channels=256,out_channels=256,kernel_size=kernel,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Conv1d(in_channels=256,out_channels=self.N_RIS2,kernel_size=kernel,stride=1),
            # nn.Tanh()
        )

    def encode_signal(self, x):
        x=self.encoder(x)
        return x

    def decode_signal(self, x):
        return self.decoder(x)
    
    def AWGN(self, x, h1_tensor, h2_tensor, D_tensor, g1_tensor, g2_tensor, phase1, phase2 ,SNR, device):
        # x = self.Nt*(x / x.norm(dim=-1)[:, None])
        SNR_linear = 10.0**(SNR/10.0)
        batch = x.shape[0]
        scale = 1e3
    
        x_compl = torch.zeros([batch,self.Nt,self.Block_length],dtype=torch.cfloat,device=device)
        for i in range(self.Nt):
            x_compl[:,i] = torch.complex(x[:,2*i],x[:,2*i+1])
            
        encode_out_compl_tensor = torch.permute(x_compl,(0,2,1))
        encode_out_compl_tensor = torch.reshape(encode_out_compl_tensor,(batch*Block_length,Nt))   
        #calculate cascaded channel    
        theta1 = torch.exp(1j*2*math.pi*phase1)
        theta2 = torch.exp(1j*2*math.pi*phase2)
        
        theta1_tensor = torch.permute(theta1,(0,2,1))
        theta1_tensor = torch.reshape(theta1_tensor,(batch*Block_length,N_RIS1))
        
        theta2_tensor = torch.permute(theta2,(0,2,1))
        theta2_tensor = torch.reshape(theta2_tensor,(batch*Block_length,N_RIS2))
        
        #calculate channel
        temp = torch.matmul(torch.diag_embed(theta1_tensor),h1_tensor)
        H1phi1D_tensor = torch.matmul(D_tensor,temp)
        
        temp = torch.matmul(torch.diag_embed(theta2_tensor),H1phi1D_tensor)
        doublelink_tensor = torch.matmul(g2_tensor,temp)
        
        temp = torch.matmul(torch.diag_embed(theta1_tensor),h1_tensor)
        H1phi1G1_tensor = torch.matmul(g1_tensor,temp)
        
        temp = torch.matmul(torch.diag_embed(theta2_tensor),h2_tensor)
        H2phi2G2_tensor = torch.matmul(g2_tensor,temp)
        
        cascaded_channel_tensor = doublelink_tensor*scale**2 + H1phi1G1_tensor*scale**2 + H2phi2G2_tensor*scale**2
        
        #transmit data via RIS
        y_compl_tensor = torch.matmul(cascaded_channel_tensor,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
        
        temp = torch.reshape(torch.view_as_real(y_compl_tensor),(batch*Block_length,2*self.Nr))
        temp = torch.reshape(temp,(batch,Block_length,2*Nr))
        decode_input = torch.permute(temp,(0,2,1))
        
        noise = torch.randn(*decode_input.size(),device=device) / ((SNR_linear) ** 0.5)

        x = decode_input + noise
        
        return x, y_compl_tensor
    def forward(self, x, h1, h2, D, g1, g2, EbNo_train):
#         EbNo_train = 1
        scale = 1e3
        device=torch.device('cuda')
        batch = x.shape[0]
        
        T1 = timeit.default_timer()
        #change data to tensor 
        h1_tensor = torch.permute(h1,(0,3,1,2))
        h1_tensor = torch.reshape(h1_tensor,(batch*Block_length,N_RIS1,Nt))
        
        h2_tensor = torch.permute(h2,(0,3,1,2))
        h2_tensor = torch.reshape(h2_tensor,(batch*Block_length,N_RIS2,Nt))
        
        D_tensor = torch.permute(D,(0,3,1,2))
        D_tensor = torch.reshape(D_tensor,(batch*Block_length,N_RIS2,N_RIS1))
        
        g1_tensor = torch.permute(g1,(0,3,1,2))
        g1_tensor = torch.reshape(g1_tensor,(batch*Block_length,Nr,N_RIS1))
        
        g2_tensor = torch.permute(g2,(0,3,1,2))
        g2_tensor = torch.reshape(g2_tensor,(batch*Block_length,Nr,N_RIS2))
        
        T2 = timeit.default_timer()
        #input for encoder
        # encode_in = torch.cat((x,cascaded_channel_real),dim=1)
        
        encode_in = x
        
        encode_out = self.encoder(encode_in)
        #power normalization
        mean = (encode_out**2).mean()
        encode_out = encode_out/torch.sqrt(2*Nt*mean)
        
        #convert encode_out to complex
        encode_out_compl = torch.zeros([batch,Nt,Block_length],dtype=torch.cfloat,device=device)
        for i in range(self.Nt):
            encode_out_compl[:,i,:] = torch.complex(encode_out[:,2*i,:],encode_out[:,2*i+1,:])
        #input for RIS1
        encode_out_compl_tensor = torch.permute(encode_out_compl,(0,2,1))
        encode_out_compl_tensor = torch.reshape(encode_out_compl_tensor,(batch*Block_length,Nt))

        y_ris1 = torch.matmul(h1_tensor,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
        y_ris1 = y_ris1*scale
        temp = torch.reshape(torch.view_as_real(y_ris1),(batch*Block_length,2*N_RIS1))
        temp = torch.reshape(temp,(batch,Block_length,2*N_RIS1))
        y_ris1_real = torch.permute(temp,(0,2,1))
            
        phase1 = self.RIS1(y_ris1_real)
        theta1 = torch.exp(1j*2*math.pi*phase1)
        
        theta1_tensor = torch.permute(theta1,(0,2,1))
        theta1_tensor = torch.reshape(theta1_tensor,(batch*Block_length,N_RIS1))
        T3 = timeit.default_timer()
        
        #input for RIS2
        temp = torch.matmul(torch.diag_embed(theta1_tensor),h1_tensor)
        H1phi1D_tensor = torch.matmul(D_tensor,temp)
        # y1_ris2 = torch.matmul(H1phi1D_tensor,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
        # y2_ris2 = torch.matmul(h2_tensor,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
        y_ris2 = torch.matmul(H1phi1D_tensor*scale+h2_tensor*scale,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
        # y_ris2 = y1_ris2 + y2_ris2
        
        temp = torch.reshape(torch.view_as_real(y_ris2),(batch*Block_length,2*self.N_RIS2))
        temp = torch.reshape(temp,(batch,Block_length,2*N_RIS2))
        y_ris2_real = torch.permute(temp,(0,2,1))
      
        phase2 = self.RIS2(y_ris2_real)
        theta2 = torch.exp(1j*2*math.pi*phase2)
      
        theta2_tensor = torch.permute(theta2,(0,2,1))
        theta2_tensor = torch.reshape(theta2_tensor,(batch*Block_length,N_RIS2))
        T4 = timeit.default_timer()
        
        #calculate cascaded channel
        temp = torch.matmul(torch.diag_embed(theta2_tensor),H1phi1D_tensor)
        doublelink_tensor = torch.matmul(g2_tensor,temp)
        
        temp = torch.matmul(torch.diag_embed(theta1_tensor),h1_tensor)
        H1phi1G1_tensor = torch.matmul(g1_tensor,temp)
        
        temp = torch.matmul(torch.diag_embed(theta2_tensor),h2_tensor)
        H2phi2G2_tensor = torch.matmul(g2_tensor,temp)
        
#         cascaded_channel_tensor = doublelink_tensor*scale**3 + H1phi1G1_tensor*scale**2  + H2phi2G2_tensor*scale**2 
        cascaded_channel_tensor = doublelink_tensor*scale**2 + H1phi1G1_tensor*scale**2  + H2phi2G2_tensor*scale**2
#         print(cascaded_channel_tensor)
#         print(g2_tensor)
        temp = torch.reshape(torch.view_as_real(cascaded_channel_tensor),(batch*Block_length,2*Nr*Nt))
        temp = torch.reshape(temp,(batch,Block_length,2*Nr*Nt))
        cascaded_channel_real = torch.permute(temp,(0,2,1))
        # cascaded_channel = torch.reshape(cascaded_channel_tensor,(batch,Block_length,Nr,Nt))
        # cascaded_channel = torch.permute(cascaded_channel,(0,2,3,1))
        T5 = timeit.default_timer()
#         print(cascaded_channel_tensor)
        #received signal
        y_compl_tensor = torch.matmul(cascaded_channel_tensor,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
#         print(y_compl_tensor)
     
#         print(y_compl_tensor)
        temp = torch.reshape(torch.view_as_real(y_compl_tensor),(batch*Block_length,2*self.Nr))
        temp = torch.reshape(temp,(batch,Block_length,2*Nr))
        decode_input = torch.permute(temp,(0,2,1))
        
        T6 = timeit.default_timer()
#         received_power = sum(torch.linalg.norm(y_compl_tensor,dim=1))
        
        #AWGN noise
        noise = Variable(torch.randn(*decode_input.size(),device=device) / ((2*(EbNo_train))**0.5))
        decode_input = decode_input + noise
        mean = (decode_input**2).mean()
#         decode_input = decode_input/torch.sqrt(2*mean)
#         channel_mean = (cascaded_channel_real**2).mean()
#         cascaded_channel_real = cascaded_channel_real/torch.sqrt(2*channel_mean)
        decode_input = torch.cat((decode_input,cascaded_channel_real),dim=1)
        decode_out = self.decoder(decode_input)
        
        T7 = timeit.default_timer()
        

        return decode_out, y_compl_tensor, noise


# Rice Correlated channel

In [3]:
#implement channel model for the wireless system

def functionDoubleScatter(NumOfTransmitAnten,NumOfReceiveAnten,NumOfScatter,alpha):
    beta= 1 #PathLoss factor
    dl = 0.5
    dS = 1
    R = 1
#     alpha = 0
    
    angularSpread = 2*math.pi/3
    theta_tilda = 0.4595
    R1 = torch.zeros(NumOfTransmitAnten,NumOfTransmitAnten,dtype=torch.cfloat) #correlation matrix between BS and scatter
    R_tilda = torch.zeros(NumOfScatter,NumOfScatter,dtype=torch.cfloat) #correlation matrix
    R2 = torch.zeros(NumOfReceiveAnten,NumOfReceiveAnten,dtype=torch.cfloat)
    
    ii = torch.arange(1-NumOfScatter,NumOfScatter,1)
    theta_array = ii*angularSpread/(NumOfScatter-1)
    theta_tilda_array = ii*2*theta_tilda/(NumOfScatter-1)
    
    for i in range(NumOfTransmitAnten):
        for j in range(NumOfTransmitAnten):
            temp = 0
            for n in range(NumOfScatter):
                temp += torch.exp(-torch.pi*2*1j*(i-j)*dl*torch.cos(torch.pi/2+alpha+theta_array[n]))
            
            R1[i,j] = 1/NumOfScatter*temp
    
    for i in range(NumOfScatter):
        for j in range(NumOfScatter):
            temp_tilda = 0
            for n in range(NumOfScatter):
                temp_tilda += torch.exp(-torch.pi*2*1j*(i-j)*dS*torch.cos(torch.pi/2+theta_tilda_array+theta_array[n]))
            
            R_tilda[i,j] = 1/NumOfScatter*temp
            
    for i in range(NumOfReceiveAnten):
        for j in range(NumOfReceiveAnten):
            temp = 0
            for n in range(NumOfScatter):
                temp += torch.exp(-torch.pi*2*1j*(i-j)*dl*torch.cos(torch.pi/2+alpha+theta_array[n]))
            R2[i,j] = 1/NumOfScatter*temp
    return R1, R_tilda, R2

def functionScatterChannelGen(NumOfTransmitAnten,NumOfReceiveAnten,NumOfScatter,R1,R_tilda,R2,gen_num):
    beta = 1
    R1_square = sqrtm(R1)
    temp = torch.tensor(R1_square,dtype=torch.cfloat).unsqueeze(0)
    R1_square = temp.repeat(gen_num,1,1)
    
    R_tilda_square = sqrtm(R_tilda)
    temp = torch.tensor(R_tilda_square,dtype=torch.cfloat).unsqueeze(0)
    R_tilda_square = temp.repeat(gen_num,1,1)
    
    R2_square = sqrtm(R2)
    temp = torch.tensor(R2_square,dtype=torch.cfloat).unsqueeze(0)
    R2_square = temp.repeat(gen_num,1,1)
    
    G = torch.randn([gen_num,NumOfScatter,NumOfTransmitAnten],dtype=torch.cfloat)
    g = torch.randn([gen_num,NumOfReceiveAnten,NumOfScatter],dtype=torch.cfloat)
    
    channel = torch.matmul(R2_square,g)
    channel = torch.matmul(channel,R_tilda_square)
    
    channel = torch.matmul(channel,G)
    channel = torch.matmul(channel,R1_square)
    channel = torch.sqrt(torch.tensor(beta/NumOfScatter))*channel
    
    return channel

def array_response_h(theta,phi,Ah,d_labmda):
    i = torch.arange(0,Ah,1)
    return torch.exp(1j*2*torch.pi*d_lambda*math.cos(theta)*math.sin(phi)*i)

def array_response_v(theta,Av,d_lambda):
    i = torch.arange(0,Av,1)
    
    return torch.exp(1j*2*torch.pi*d_lambda*math.sin(theta)*i)

def UPA_response(theta,phi,Av,Ah,d_lambda):
    a_v = array_response_v(theta,Av,d_lambda)
    a_h = array_response_h(theta,phi,Ah,d_lambda)
    
    return torch.kron(a_v,a_h)

def ULA_response(theta,N,d_lamda):
    i = torch.arange(0,N,1)
    
    return torch.exp(1j*2*torch.pi*d_lambda*math.sin(theta)*i)

def correlated_channel(N_t,N_r,rho_t,rho_r,gen_num):
    h = torch.randn([gen_num,N_r,N_t],dtype=torch.cfloat)
    correlated_channel = torch.zeros(gen_num,N_r,N_t,dtype=torch.cfloat)
    R1 = torch.zeros(N_r,N_r)
    R2 = torch.zeros(N_t,N_t)
    for i in range(N_r):
        for j in range(N_r):
            R1[i,j] = pow(rho_r,abs(i-j))
    for i in range(N_t):
        for j in range(N_t):
            R2[i,j] = pow(rho_t,abs(i-j))
    R1_square = sqrtm(R1)
    temp = torch.tensor(R1_square,dtype=torch.cfloat).unsqueeze(0)
    R1_square = temp.repeat(gen_num,1,1)
    R2_square = sqrtm(R2)
    temp = torch.tensor(R2_square,dtype=torch.cfloat).unsqueeze(0)
    R2_square = temp.repeat(gen_num,1,1)
    
    temp = torch.matmul(R1_square,h)
    correlated_channel = torch.matmul(temp,R2_square)
    return correlated_channel

def Rice_channel(K,theta_A,phi_A,Nr_v,Nr_h,Nr,rho_r,theta_D,phi_D,Nt_v,Nt_h,Nt,rho_t,d_lambda,gen_num,d,num_scatter,alpha,mode):
    sigma = 1e-12
#     H_tilda = correlated_channel(Nt,Nr,rho_t,rho_r,gen_num) #correlated channel
    R1, R_tilda, R2 = functionDoubleScatter(Nt,Nr,num_scatter,alpha)
    H_tilda = functionScatterChannelGen(Nt,Nr,num_scatter,R1,R_tilda,R2,gen_num) #double_scatter channel
    if mode == 'D':
        H_bar = torch.matmul(torch.transpose(torch.conj(UPA_response(theta_A,phi_A,Nr_v,Nr_h,d_lambda).unsqueeze(0)),0,1),UPA_response(theta_D,phi_D,Nt_v,Nt_h,d_lambda).unsqueeze(0))
    elif mode == 'H':
        H_bar = torch.matmul(torch.transpose(torch.conj(UPA_response(theta_A,phi_A,Nr_v,Nr_h,d_lambda).unsqueeze(0)),0,1),ULA_response(theta_D,Nt,d_lambda).unsqueeze(0))
    elif mode == 'G':
        H_bar = torch.matmul(torch.transpose(torch.conj(ULA_response(theta_A,Nr,d_lambda).unsqueeze(0)),0,1),UPA_response(theta_D,phi_D,Nt_v,Nt_h,d_lambda).unsqueeze(0))
    temp = H_bar.unsqueeze(0)
    H_bar = temp.repeat(gen_num,1,1)
    # F = 4*torch.randn(1)
    
#     beta_db = -34.53 -38*math.log10(d);
    beta_db = -35.6 -22*math.log10(d);
    beta = pow(10,beta_db/10);
#     beta = beta/(sigma)
    H = math.sqrt(beta)*(math.sqrt(K/(K+1))*H_bar + math.sqrt(1/(K+1))*H_tilda)
    return H

# System Parameter

In [4]:
d1 = 100 #BS --> RIS distance

d2 = 200 #BS --> user distance

d = math.sqrt(d1**2+d2**2) #diag distance
h = 2;
# d3 = 25 #receiver --> BS horizontal distance

# d_RIS_user = math.sqrt((d0-d)**2+dv**2)

alpha = 3.5 #pathloss exponent

Av = 8 #RIS vertical size

Ah = 4 #RIS horizontal size

d_lambda = 0.5 #d:lambda ratio

#AoA/AoD angles
#BS --> RIS1
theta_A_TR1 = torch.pi/4
theta_D_TR1 = torch.pi/2
phi_A_TR1 = 0

#BS to RIS2
theta_A_TR2 = abs(torch.pi/4 - math.atan(d2/d1))
theta_D_TR2 = math.atan(d1/d2)
phi_A_TR2 = 0

#RIS1 --> RIS2

theta_A_R1R2 = torch.pi/4
theta_D_R1R2 = torch.pi/4
phi_A_R1R2 = 0
phi_D_R1R2 = 0

#RIS1 --> user
theta_A_R1R = math.atan(d2/d1)
theta_D_R1R = math.atan(d1/d2)
phi_D_R1R = math.atan(h/math.sqrt(d1**2+d2**2))


#RIS2 --> user
theta_A_R2R = 0 
theta_D_R2R = torch.pi/4
phi_D_R2R = math.atan(h/d1)


#RIS1 size
K1_v = 4;
K1_h = 8;
K1 = K1_v*K1_h;
N_RIS1 = K1;

#RIS2 size
K2_v = 4;
K2_h = 8;
K2 = K2_v*K2_h;
N_RIS2 = K2;

#BS antenna size
Nt_v = 4
Nt_h = 4
Nt = Nt_v*Nt_h;

#user antenna size
Nr_v = 4
Nr_h = 4
Nr = Nr_v*Nr_h;

#Rice parameters
K = 0.2

rho_t = 0.6
rho_r = 0.6
rho_RIS = 0.6

num_scatter = 3
#level of modulation
N_bit = 64
sigma = 1e-12
gen_num = 10000
Block_length = 20
train_ratio = 0.9
train_num = int(train_ratio*(gen_num/Block_length))
BATCH_SIZE = 128


In [ ]:
#Generate channel samples

h1_init = Rice_channel(K,theta_A_TR1,phi_A_TR1,K1_v,K1_h,N_RIS1,rho_RIS,theta_D_TR1,0,Nt_v,Nt_h,Nt,rho_t,d_lambda,gen_num,d1,num_scatter,0,'H')
h2_init = Rice_channel(K,theta_A_TR2,phi_A_TR2,K2_v,K2_h,N_RIS2,rho_RIS,theta_D_TR2,0,Nt_v,Nt_h,Nt,rho_t,d_lambda,gen_num,d,num_scatter,0,'H')
D_init = Rice_channel(K,theta_A_R1R2,phi_A_R1R2,K2_v,K2_h,N_RIS2,rho_RIS,theta_D_R1R2,phi_D_R1R2,K1_v,K1_h,N_RIS1,rho_t,d_lambda,gen_num,d2,num_scatter,0,'D')
g1_init = Rice_channel(K,theta_A_R1R,0,Nr_v,Nr_h,Nr,rho_r,theta_D_R1R,phi_D_R1R,K1_v,K1_h,N_RIS1,rho_t,d_lambda,gen_num,d,num_scatter,torch.pi/12,'G')
g2_init = Rice_channel(K,theta_A_R2R,0,Nr_v,Nr_h,Nr,rho_r,theta_D_R2R,phi_D_R2R,K2_v,K2_h,N_RIS2,rho_t,d_lambda,gen_num,d1,num_scatter,torch.pi/12,'G')

In [ ]:
batch = int(gen_num/Block_length)

temp = torch.reshape(h1_init,(batch,Block_length,N_RIS1,Nt))
h1_all = torch.permute(temp,(0,2,3,1))

temp = torch.reshape(h2_init,(batch,Block_length,N_RIS2,Nt))
h2_all = torch.permute(temp,(0,2,3,1))

temp = torch.reshape(D_init,(batch,Block_length,N_RIS2,N_RIS1))
D_all = torch.permute(temp,(0,2,3,1))

temp = torch.reshape(g1_init,(batch,Block_length,Nr,N_RIS1))
g1_all = torch.permute(temp,(0,2,3,1))

temp = torch.reshape(g2_init,(batch,Block_length,Nr,N_RIS2))
g2_all = torch.permute(temp,(0,2,3,1))

In [ ]:
batch_train = int(gen_num/Block_length*train_ratio)
repeat_num = 2

temp = h1_all[:batch_train,:,:,:Block_length]
temp = temp.repeat(repeat_num,1,1,1)
inx = torch.randperm(train_num)
h1_train = temp[inx,:]

temp = h2_all[:batch_train,:,:,:Block_length]
temp = temp.repeat(repeat_num,1,1,1)
inx = torch.randperm(train_num)
h2_train = temp[inx,:]

temp = D_all[:batch_train,:,:,:Block_length]
temp = temp.repeat(repeat_num,1,1,1)
inx = torch.randperm(train_num)
D_train = temp[inx,:]

temp = g1_all[:batch_train,:,:,:Block_length]
temp = temp.repeat(repeat_num,1,1,1)
inx = torch.randperm(train_num)
g1_train = temp[inx,:]

temp = g2_all[:batch_train,:,:,:Block_length]
temp = temp.repeat(repeat_num,1,1,1)
inx = torch.randperm(train_num)
g2_train = temp[inx,:]

del inx
del temp




In [ ]:
#Create training sample
train_label = (torch.rand([train_num,Block_length]) * N_bit).long()
train_data = torch.zeros(train_num,N_bit,Block_length)
for i in range(train_num):
    train_data[i,:] = torch.transpose(torch.sparse.torch.eye(N_bit).index_select(dim=0, index=train_label[i,:]),0,1)
    
dataset = Data.TensorDataset(train_data,train_label, h1_train, h2_train, D_train, g1_train, g2_train)
train_loader = Data.DataLoader(dataset = dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available()==True else 'cpu')
model = CNN_autoencoder(N_bit, Nt, Nr, N_RIS1, N_RIS2, Block_length).to(device)
#model training parameters
optimizer = Adam(model.parameters(),lr=0.001)
loss_fn = torch.nn.BCELoss()
# loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
model.load_state_dict(torch.load('Autoencoder_BLength20_16QAM_Rice1_Scatter3_MIMO16x16_train0dB.pt'))

In [ ]:

optimizer = Adam(model.parameters(),lr=0.00005)

In [ ]:
#training
num_epoch = 100
model = model.to(device)
SNR_train = 20#db
SNR_linear = 10**(SNR_train/10) #linear

t0 = time.perf_counter()
for epoch in range(num_epoch):

    for step, (data_in,label,h1,h2,D,g1,g2), in enumerate(train_loader): 
        with torch.autograd.set_detect_anomaly(True):
            batch = data_in.shape[0]
            data_in = data_in.to(device)
            h1 = h1.to(device)
            h2 = h2.to(device)
            D = D.to(device)
            g1 = g1.to(device)
            g2 = g2.to(device)

#             decoded, received_power = model(data_in,h1,h2,D,g1,g2,E0Nb_train)
            decoded, y, noise = model(data_in,h1,h2,D,g1,g2,SNR_linear)
#             received_power = sum(torch.linalg.norm(y,dim=1))
#             received_power = (received_power/batch/Block_length)
            loss = loss_fn(decoded, data_in)

            optimizer.zero_grad()               # clear gradients for this training step
            loss.backward()                     # backpropagation, compute gradients
            optimizer.step()  
            decode_out = decoded.cpu()
            decode_out = decode_out.data.numpy()

            position = np.argmax(decode_out,axis=1)
            error = label.data.numpy() != position
            num_error = error.astype(int).sum()
            Ber = num_error/(batch*Block_length)
            if step % 1 == 0:
#                 print('Epoch: ',epoch, '|train loss: %.4f' %(loss.data),' BER: %.5f' %Ber, 'received_power: %3f' %received_power)
                print('Epoch: ',epoch, '|train loss: %.4f' %(loss.data),' BER: %.5f' %Ber)
            
t1 = time.perf_counter()


In [ ]:
torch.save(model.state_dict(),'Autoencoder_BLength20_16QAM_Rice1_Scatter3_MIMO16x16_train15dB.pt')

# Testing phase


In [ ]:
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump

In [ ]:
import matplotlib.pyplot as plt
model = model.to(device)


test_num = 1000
test_label = (torch.rand([test_num,Block_length]) * N_bit).long()
test_data = torch.zeros(test_num,N_bit,Block_length)
channel_error = 0.0
scale = 1e3
for i in range(test_num):
    test_data[i,:] = torch.transpose(torch.sparse.torch.eye(N_bit).index_select(dim=0, index=test_label[i,:]),0,1)
    
h1 = h1_train[:test_num,:]
h2 = h2_train[:test_num,:]
D = D_train[:test_num,:]
g1 = g1_train[:test_num,:]
g2 = g2_train[:test_num,:]


EbNodB_range = list(frange(-10,21,2))
Ber = torch.zeros(len(EbNodB_range))
test_label = test_label.data.numpy()
test_data = test_data.to(device)
for n in range(0,len(EbNodB_range)):
    h1 = h1.to(device)
    h2 = h2.to(device)
    D = D.to(device)
    g1 = g1.to(device)
    g2 = g2.to(device)
    
    h1_tensor = torch.permute(h1,(0,3,1,2))
    h1_tensor = torch.reshape(h1_tensor,(test_num*Block_length,N_RIS1,Nt))
    
    h2_tensor = torch.permute(h2,(0,3,1,2))
    h2_tensor = torch.reshape(h2_tensor,(test_num*Block_length,N_RIS2,Nt))
    
    D_tensor = torch.permute(D,(0,3,1,2))
    D_tensor = torch.reshape(D_tensor,(test_num*Block_length,N_RIS2,N_RIS1))
    
    g1_tensor = torch.permute(g1,(0,3,1,2))
    g1_tensor = torch.reshape(g1_tensor,(test_num*Block_length,Nr,N_RIS1))
    
    g2_tensor = torch.permute(g2,(0,3,1,2))
    g2_tensor = torch.reshape(g2_tensor,(test_num*Block_length,Nr,N_RIS2))
    
    encode_in = test_data
    encode_out = model.encoder(encode_in)
    mean = (encode_out**2).mean()
    encode_out = encode_out / torch.sqrt(2*Nt*mean)
    
    encode_out_compl = torch.zeros([test_num,Nt,Block_length],dtype=torch.cfloat,device=device)
    for i in range(Nt):
        encode_out_compl[:,i,:] = torch.complex(encode_out[:,2*i,:],encode_out[:,2*i+1,:])
    
    #input for RIS1 
    encode_out_compl_tensor = torch.permute(encode_out_compl,(0,2,1))
    encode_out_compl_tensor = torch.reshape(encode_out_compl_tensor,(test_num*Block_length,Nt))
        
    y_ris1 = torch.matmul(h1_tensor,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
    y_ris1 = y_ris1*scale
    temp = torch.reshape(torch.view_as_real(y_ris1),(test_num*Block_length,2*N_RIS1))
    temp = torch.reshape(temp,(test_num,Block_length,2*N_RIS1))
    y_ris1_real = torch.permute(temp,(0,2,1))
        
    phase1 = model.RIS1(y_ris1_real)
#     phase = torch.randn(test_num,N_RIS,Block_length,device=device)
    theta1 = torch.exp(1j*2*math.pi*phase1)
    theta1_tensor = torch.permute(theta1,(0,2,1))
    theta1_tensor = torch.reshape(theta1_tensor,(test_num*Block_length,N_RIS1))
    
    #input for RIS2
    temp = torch.matmul(torch.diag_embed(theta1_tensor),h1_tensor)
    H1phi1D_tensor = torch.matmul(D_tensor,temp)
    y_ris2 = torch.matmul(H1phi1D_tensor*scale+h2_tensor*scale,encode_out_compl_tensor.unsqueeze(2)).squeeze(2)
    
    temp = torch.reshape(torch.view_as_real(y_ris2),(test_num*Block_length,2*N_RIS2))
    temp = torch.reshape(temp,(test_num,Block_length,2*N_RIS2))
    y_ris2_real = torch.permute(temp,(0,2,1))
  
    phase2 = model.RIS2(y_ris2_real)
    theta2 = torch.exp(1j*2*math.pi*phase2)
      
    theta2_tensor = torch.permute(theta2,(0,2,1))
    theta2_tensor = torch.reshape(theta2_tensor,(test_num*Block_length,N_RIS2))
    
#calculate cascaded channel
    temp = torch.matmul(torch.diag_embed(theta2_tensor),H1phi1D_tensor)
    doublelink_tensor = torch.matmul(g2_tensor,temp)
    
    temp = torch.matmul(torch.diag_embed(theta1_tensor),h1_tensor)
    H1phi1G1_tensor = torch.matmul(g1_tensor,temp)
    
    temp = torch.matmul(torch.diag_embed(theta2_tensor),h2_tensor)
    H2phi2G2_tensor = torch.matmul(g2_tensor,temp)
    
    cascaded_channel_tensor = doublelink_tensor*scale**2 + H1phi1G1_tensor*scale**2 + H2phi2G2_tensor*scale**2
    temp = torch.reshape(torch.view_as_real(cascaded_channel_tensor),(test_num*Block_length,2*Nr*Nt))
    temp = torch.reshape(temp,(test_num,Block_length,2*Nr*Nt))
    cascaded_channel_real = torch.permute(temp,(0,2,1))
    noise = (torch.randn([test_num,2*Nt*Nr,Block_length],device=device)/math.sqrt(2))*channel_error
    cascaded_channel_real = cascaded_channel_real 
    
    decode_in, y_compl = model.AWGN(encode_out,h1_tensor, h2_tensor, D_tensor, g1_tensor, g2_tensor, phase1, phase2, EbNodB_range[n],device)
   
    
    # mean = (decode_in**2).mean()
    # decode_in = decode_in/torch.sqrt(2*mean)
#     channel_mean = (cascaded_channel_real**2).mean()
#     cascaded_channel_real = cascaded_channel_real/torch.sqrt(2*channel_mean)
    noise_data = torch.randn(decode_in.shape)
    decode_in_concat = torch.cat((decode_in,cascaded_channel_real),dim=1)
    decode_out = model.decoder(decode_in_concat)
    decode_out = decode_out.cpu()
    decode_out = decode_out.data.numpy()
    
    position = np.argmax(decode_out,axis=1)
    error = test_label != position
    num_error = error.astype(int).sum()
    Ber[n] += num_error/(test_num*Block_length)
    
    print("SNR {}: Ber = {}".format(EbNodB_range[n],Ber[n]))

import matplotlib.pyplot as plt
plt.plot(EbNodB_range, Ber, 'bo',label='Autoencoder(4,4)')
plt.yscale('log')
plt.xlabel('SNR Range')
plt.ylabel('Block Error Rate')
plt.grid()


In [ ]:
model.load_state_dict(torch.load('Autoencoder_BLength20_16QAM_Rice1_Scatter3_MIMO16x16_train10dB.pt'))

In [ ]:
ser = Ber.tolist()
from scipy.io import savemat
mdic = {"ser": ser}
savemat("SER_Auto_Blength20_16x16_64RIS_64QAM_train20dB_3Scatter_Rice.2.mat",mdic)